## Computing 8 secondary emotions 

In [3]:
# Load packages
!pip install pandas
!pip install matplotlib
!pip install numpy scipy statsmodels

import pandas as pd
import json
import matplotlib.pyplot as plt


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Original emotions dataset

In [4]:
# Loading data
df = pd.read_csv('/work/VNV/Module 3 - Emotion analysis/data/original_emotions_dataset.csv')
df

,lineID,characterID,movieID,character_name,text,gender,movie_year,emotions,top_emotion
0,L1045,u0,m0,BIANCA,They do not!,f,1999,"{'sadness': 0.008004407398402691, 'joy': 0.010...",anger
1,L1044,u2,m0,CAMERON,They do to!,m,1999,"{'sadness': 0.007097005378454924, 'joy': 0.046...",anger
2,L985,u0,m0,BIANCA,I hope so.,f,1999,"{'sadness': 0.0004969622241333127, 'joy': 0.99...",joy
3,L984,u2,m0,CAMERON,She okay?,m,1999,"{'sadness': 0.010399539023637772, 'joy': 0.958...",joy
4,L925,u0,m0,BIANCA,Let's go.,f,1999,"{'sadness': 0.019666552543640137, 'joy': 0.945...",joy
...,...,...,...,...,...,...,...,...,...
241577,L665696,u9019,m615,IGOR,Whew! A nineteen-inch drill.,m,1974,"{'sadness': 0.002676282776519656, 'joy': 0.141...",anger
241578,L665695,u9020,m615,INGA,His schwanzstucker.,f,1974,"{'sadness': 0.023279836401343346, 'joy': 0.034...",anger
241579,L665694,u9019,m615,IGOR,His what??,m,1974,"{'sadness': 0.01117276307195425, 'joy': 0.0121...",anger
241580,L665652,u9020,m615,INGA,There must have been someone else down here then.,f,1974,"{'sadness': 0.054072778671979904, 'joy': 0.010...",fear


#### Preprocessing the dataset

In [5]:
# Select relevant columns
#df = df[["gender", "text", "emotions"]]

# Replace values in the 'gender' column
df['gender'] = df['gender'].replace({'f': 'female', 'F': 'female', 'm': 'male', 'M': 'male'})

# Inspect the structure of emotions
print(df['emotions'].apply(type).unique()) # class: str

# Convert emotion strings to dictionaries
df['emotions'] = df['emotions'].apply(lambda x: json.loads(x.replace("'", '"')))

# Check if the dictionaries have the same structure for all rows
keys_list = df['emotions'].apply(lambda x: list(x.keys())) # extract keys from all dictionaries
keys_list_as_tuples = keys_list.apply(tuple) # convert the list of keys into a tuple to be able to use nunique() function
print(keys_list_as_tuples.nunique())  # number of unique key structures is 1 

# Convert the emotions dictionary column into separate columns for each emotion
emotions_df = pd.json_normalize(df['emotions'])

# Add the emotions columns to the original DataFrame
df = pd.concat([df.drop(columns=['emotions']), emotions_df], axis=1)

# Inspect the values of each emotion
print(df.describe())


[<class 'str'>]
1
          movie_year        sadness            joy           love  \
count  241582.000000  241582.000000  241582.000000  241582.000000   
mean     1988.020349       0.086657       0.390462       0.019333   
std        17.241337       0.210234       0.405271       0.093762   
min      1927.000000       0.000106       0.000114       0.000121   
25%      1984.000000       0.003691       0.012251       0.001281   
50%      1995.000000       0.012598       0.183366       0.003133   
75%      1999.000000       0.043087       0.875908       0.007239   
max      2010.000000       0.998856       0.999103       0.996130   

               anger           fear       surprise  
count  241582.000000  241582.000000  241582.000000  
mean        0.344858       0.146999       0.011690  
std         0.356802       0.253918       0.063436  
min         0.000048       0.000057       0.000116  
25%         0.015238       0.002841       0.001111  
50%         0.192871       0.020284       

#### Compute 8 secondary emotions

In [6]:
df['guilt'] = df['joy'] * df['fear']
df['despair'] = df['fear'] * df['sadness']
df['disapproval'] = df['surprise'] * df['sadness']
df['envy'] = df['sadness'] * df['anger']
df['delight'] = df['joy'] * df['surprise']
df['pride'] = df['anger'] * df['joy']
df['outrage'] = df['surprise'] * df['anger']
df['awe'] = df['fear'] * df['surprise']


#### Normalize the probabilities
 we calculate the sum of all emotion columns (including the 6 original and 8 new ones) and divide each emotion by the total sum for that row. This will normalize the probabilities so they still sum to 1.

In [7]:
# Combine the original columns and the new ones
all_emotions = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise', 'guilt', 'despair', 'disapproval', 'envy', 'delight', 'pride', 'outrage', 'awe']

# Normalize the columns by dividing each value by the total sum for that row
df[all_emotions] = df[all_emotions].div(df[all_emotions].sum(axis=1), axis=0)

# Verifying that the sum of each row is 1
df[all_emotions].sum(axis=1)

# Saving the new dataset with 14 emotions
df.to_csv('/work/VNV/Module 3 - Emotion analysis/data/original_14_emotions_dataset.csv', index=False)

## Create a column for the 14 dimensional vector

In [14]:
# List of basic emotion columns in the order we want the vector
basic_emotions_columns = [
    "sadness", "joy", "love", "anger", "fear", "surprise"
]
# List of all emotions
all_emotions_columns = [
    "sadness", "joy", "love", "anger", "fear", "surprise",
    "guilt", "despair", "disapproval", "envy", "delight",
    "pride", "outrage", "awe"
]

# Create a new column `basic_emotions_vector` with a list of emotion values per row
df["basic_emotions_vector"] = df[basic_emotions_columns].values.tolist()

# Create a new column `all_emotions_vector` with a list of emotion values per row
df["all_emotions_vector"] = df[all_emotions_columns].values.tolist()

# Display the updated DataFrame (optional)
print(df.head())

# Saving the new dataset with 14 emotions
df.to_csv('/work/VNV/Module 4 - Clustering and prediction/data/original_14_emotions_dataset.csv', index=False)

   lineID characterID movieID character_name          text  gender  \
0  L1045           u0      m0         BIANCA  They do not!  female   
1  L1044           u2      m0        CAMERON   They do to!    male   
2   L985           u0      m0         BIANCA    I hope so.  female   
3   L984           u2      m0        CAMERON     She okay?    male   
4   L925           u0      m0         BIANCA     Let's go.  female   

   movie_year top_emotion   sadness       joy  ...     guilt       despair  \
0        1999       anger  0.007853  0.010045  ...  0.000099  7.728175e-05   
1        1999       anger  0.006743  0.043789  ...  0.000429  6.599628e-05   
2        1999         joy  0.000497  0.991094  ...  0.000198  9.931532e-08   
3        1999         joy  0.010124  0.933074  ...  0.020098  2.180714e-04   
4        1999         joy  0.019116  0.919320  ...  0.011252  2.339689e-04   

    disapproval          envy   delight     pride       outrage           awe  \
0  1.148060e-05  7.612334e-03

## Genderize emotion dataset

In [16]:
df_1 = pd.read_csv('/work/VNV/Module 3 - Emotion analysis/data/genderize_emotions_dataset.csv') 
df_1

,lineID,characterID,movieID,character_name,text,gender,movie_year,emotions,top_emotion
0,L1045,u0,m0,BIANCA,They do not!,f,1999,"{'sadness': 0.008004426956176758, 'joy': 0.010...",anger
1,L1044,u2,m0,CAMERON,They do to!,m,1999,"{'sadness': 0.007097034249454737, 'joy': 0.046...",anger
2,L985,u0,m0,BIANCA,I hope so.,f,1999,"{'sadness': 0.0004969646688550711, 'joy': 0.99...",joy
3,L984,u2,m0,CAMERON,She okay?,m,1999,"{'sadness': 0.01039950642734766, 'joy': 0.9584...",joy
4,L925,u0,m0,BIANCA,Let's go.,f,1999,"{'sadness': 0.019666505977511406, 'joy': 0.945...",joy
...,...,...,...,...,...,...,...,...,...
277684,L666262,u9031,m616,MELVILL,Lighting COGHILL' 5 cigar: Our good Colonel Du...,f,1979,"{'sadness': 0.0021832550410181284, 'joy': 0.98...",joy
277685,L666521,u9030,m616,DURNFORD,"And I assure you, you do not In fact I'd be ob...",m,1979,"{'sadness': 0.006201059557497501, 'joy': 0.622...",joy
277686,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...,m,1979,"{'sadness': 0.007595515809953213, 'joy': 0.217...",anger
277687,L666369,u9030,m616,DURNFORD,"Your orders, Mr Vereker?",m,1979,"{'sadness': 0.008677853271365166, 'joy': 0.013...",anger


### Preprocessing the dataset

In [17]:
# Select relevant columns
#df_1= df_1[["gender", "text", "emotions"]]

# Replace values in the 'gender' column
df_1['gender'] = df_1['gender'].replace({'f': 'female', 'F': 'female', 'm': 'male', 'M': 'male'})

# Inspect the structure of emotions
print(df_1['emotions'].apply(type).unique()) # class: str

# Convert emotion strings to dictionaries
df_1['emotions'] = df_1['emotions'].apply(lambda x: json.loads(x.replace("'", '"')))

# Check if the dictionaries have the same structure for all rows
keys_list = df_1['emotions'].apply(lambda x: list(x.keys())) # extract keys from all dictionaries
keys_list_as_tuples = keys_list.apply(tuple) # convert the list of keys into a tuple to be able to use nunique() function
print(keys_list_as_tuples.nunique())  # number of unique key structures is 1 

# Convert the emotions dictionary column into separate columns for each emotion
emotions_df_1= pd.json_normalize(df_1['emotions'])

# Add the emotions columns to the original DataFrame
df_1 = pd.concat([df_1.drop(columns=['emotions']), emotions_df_1], axis=1)

# Inspect the values of each emotion
print(df_1.describe())


[<class 'str'>]
1
          movie_year        sadness            joy           love  \
count  277689.000000  277689.000000  277689.000000  277689.000000   
mean     1988.157414       0.086277       0.390500       0.019221   
std        17.154466       0.209930       0.405205       0.093273   
min      1927.000000       0.000106       0.000114       0.000121   
25%      1984.000000       0.003683       0.012263       0.001281   
50%      1995.000000       0.012555       0.183416       0.003139   
75%      1999.000000       0.042680       0.875859       0.007243   
max      2010.000000       0.998856       0.999103       0.996130   

               anger           fear       surprise  
count  277689.000000  277689.000000  277689.000000  
mean        0.345940       0.146454       0.011608  
std         0.357190       0.253436       0.063102  
min         0.000048       0.000057       0.000116  
25%         0.015397       0.002844       0.001111  
50%         0.194549       0.020177       

### Compute 8 secondary emotions

In [18]:
df_1['guilt'] = df_1['joy'] * df_1['fear']
df_1['despair'] = df_1['fear'] * df_1['sadness']
df_1['disapproval'] = df_1['surprise'] * df_1['sadness']
df_1['envy'] = df_1['sadness'] * df_1['anger']
df_1['delight'] = df_1['joy'] * df_1['surprise']
df_1['pride'] = df_1['anger'] * df_1['joy']
df_1['outrage'] = df_1['surprise'] * df_1['anger']
df_1['awe'] = df_1['fear'] * df_1['surprise']


### Normalize probabilities

In [19]:
# Combine the original columns and the new ones
all_emotions = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise', 'guilt', 'despair', 'disapproval', 'envy', 'delight', 'pride', 'outrage', 'awe']

# Normalize the columns by dividing each value by the total sum for that row
df_1[all_emotions] = df_1[all_emotions].div(df_1[all_emotions].sum(axis=1), axis=0)

# Verifying that the sum of each row is 1
df_1[all_emotions].sum(axis=1)

# Saving the new dataset with 14 emotions
df_1.to_csv('/work/VNV/Module 3 - Emotion analysis/data/genderize_14_emotions_dataset.csv', index=False)

## Creating vectors for basic emotions and all emotions

In [21]:
# List of basic emotion columns in the order we want the vector
basic_emotions_columns = [
    "sadness", "joy", "love", "anger", "fear", "surprise"
]
# List of all emotions
all_emotions_columns = [
    "sadness", "joy", "love", "anger", "fear", "surprise",
    "guilt", "despair", "disapproval", "envy", "delight",
    "pride", "outrage", "awe"
]

# Create a new column `basic_emotions_vector` with a list of emotion values per row
df_1["basic_emotions_vector"] = df_1[basic_emotions_columns].values.tolist()

# Create a new column `all_emotions_vector` with a list of emotion values per row
df_1["all_emotions_vector"] = df_1[all_emotions_columns].values.tolist()


# Saving the new dataset with 14 emotions
df_1.to_csv('/work/VNV/Module 4 - Clustering and prediction/data/genderize_14_emotions_dataset.csv', index=False)
